Upload files to colab

Translation function

In [0]:
#!/usr/bin/env python
# coding: utf-8

# Translate sentences to English and save mapping dicitionary as json.

# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

# Parameters:
# *   sentences =
# *   dictpath =  json dictionary file mapping original sentence with translation
# 
# 

# In[ ]:


from google.cloud import translate_v2 as translate
from os.path import isfile
from os import environ
import six
import json

def translate2dict(sentences, dictpath):
    environ['GOOGLE_APPLICATION_CREDENTIALS'] = './key.json'
    translate_client = translate.Client()

    def translation(text):
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        result = translate_client.translate(text,'pt-br')

        return result['translatedText']

    if not isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    for item in sentences:
        with open(dictpath) as f:
            translations = json.load(f)
        
        if item not in translations.keys():
            translations[item] = translation(item)
            
            with open(dictpath, 'w+') as f:
                json.dump(translations, f)

Prepare input

In [3]:
from pprint import pprint
import pandas as pd
import os

sentences = list()
label_idx = [
             'sentence1',
             'sentence2',
            ]
length = 0

for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        print(label)
        print('\n'*2)

        sentences.extend(list(label))
        length = length + label.size


assert length == len(sentences)
sentences = set(sentences)

train.tsv
0                                  A plane is taking off.
1                         A man is playing a large flute.
2           A man is spreading shreded cheese on a pizza.
3                            Three men are playing chess.
4                             A man is playing the cello.
                              ...                        
5744           Severe Gales As Storm Clodagh Hits Britain
5745    Dozens of Egyptians hostages taken by Libyan t...
5746                         President heading to Bahrain
5747           China, India vow to further bilateral ties
5748     Putin spokesman: Doping charges appear unfounded
Name: sentence1, Length: 5749, dtype: object



0                             An air plane is taking off.
1                               A man is playing a flute.
2       A man is spreading shredded cheese on an uncoo...
3                              Two men are playing chess.
4                      A man seated is playing the cello.
              

Translate

In [0]:
translate2dict(sentences,  'stsb.json')

Replace with translation

In [0]:
from os import mkdir
import json

with open('stsb.json') as reader:
    dictionary = json.load(reader)

mkdir('translation')
for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', index = None, sep = '\t')